In [1]:
import sys
# !{sys.executable} -m pip install opencv-python
from IPython.core.debugger import set_trace
import cv2 
import numpy as np
from scipy import signal
from IPython.core.debugger import set_trace
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

thresh = 127


In [2]:
def find_dft(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
#     gray = cv2.threshold(gray, thresh, 255, cv2.THRESH_BINARY)[1]
    edged = cv2.Canny(gray, 100, 200)  #30,200
    contours, hierarchy = cv2.findContours(edged,  
        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
#     print(np.shape(contours))
    if np.shape(contours)[0] > 1:
        contours = contours[0]
#         contours = np.concatenate((contours[0],contours[1]))
        contours = np.expand_dims(contours, axis=0)
    if np.shape(contours)[0]!=1:
        return [],[],False
    contours1 = np.reshape(contours,[-1,2])
    contours1 = np.divide(contours1,np.repeat(np.reshape(np.add(np.power(contours1[:,0],2),np.power(contours1[:,1],2)),[-1,1]),2,axis = 1))

    u_hat = contours1[:,0]+1j*contours1[:,1]
    u_hat[~np.isnan(u_hat).any(axis=0)]
    template = np.fft.fft(u_hat)

    return template,contours,True

image = cv2.imread('./sample.png') 
targetImage = cv2.imread('./a3.png') 

sheight,swidth = image.shape[0],image.shape[1]
height,width = targetImage.shape[0],targetImage.shape[1]

cv2.waitKey(0) 
  
# Grayscale 
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
# gray = cv2.threshold(gray, thresh, 255, cv2.THRESH_BINARY)[1]
  
# Find Canny edges 
edged = cv2.Canny(gray, 100, 200) 
cv2.waitKey(0) 
  
# Finding Contours 
# Use a copy of the image e.g. edged.copy() 
# since findContours alters the image 
contours, hierarchy = cv2.findContours(edged,  
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 

template,_,_ = find_dft(image)
# template_abs = np.absolute(template)
# index = sorted(np.argsort(template_abs)[:50])
# template  = template[index]

# template_unit = np.divide(template,np.mean(np.absolute(template)))[:80]
# cv2.imshow('Canny Edges After Contouring', edged) 
# cv2.waitKey(0) 

print("Number of Contours found = " + str(len(contours))) 
  
# Draw all contours 
# -1 signifies drawing all contours 
# cv2.drawContours(image, contours, -1, (0, 255, 0), 1) 
  
# cv2.imshow('Contours', image) 
# cv2.waitKey(0) 
# cv2.destroyAllWindows() 



Number of Contours found = 1
Number of Contours found = 1


In [3]:
# for h in range(sheight,height,5):
#     for w in range(swidth,width,5):
#         crop = targetImage[h-sheight:h,w-swidth:w]
#         stemplate,contour,checker = find_dft(crop)
# #         cv2.imshow('crops', crop) 
# #         cv2.waitKey(0) 
# #         cv2.destroyAllWindows() 
#         if checker and len(stemplate)>=temp_length and len(stemplate)>=80:
#             stemplate_abs = np.absolute(stemplate)
#             index1 = sorted(np.argsort(stemplate_abs)[:80])
#             stemplate_unit = np.divide(stemplate,np.absolute(stemplate))
#             stemplate_unit  = stemplate_unit[index1]
# #             dist = np.mean(np.power(np.absolute(template_unit - stemplate_unit),2))
#             dist = np.mean(np.power(np.imag(template_unit)-np.imag(stemplate_unit),2)+np.power(np.real(template_unit)-np.real(stemplate_unit),2))
# #             set_trace()
#             if dist:
#                 cv2.drawContours(crop, contour, -1, (0, 255, 0), 1) 
#                 cv2.imshow('crops', crop) 
#                 cv2.waitKey(0) 
#                 cv2.destroyAllWindows() 
#                 print(dist)
#         else:
#             continue
   

In [4]:
# x = []
# y = []

# for h in range(sheight,height,2):
#     for w in range(swidth,width,2):
#         crop = targetImage[h-sheight:h,w-swidth:w]
#         stemplate,contour,checker = find_dft(crop)
        
# #         if checker and len(stemplate)>=temp_length and len(stemplate)>=80:
#         if checker and len(contour)>0 and len(stemplate)>=50:
#             stemplate = stemplate[:50]
# #             dist1 = np.mean(np.power(np.imag(template)-np.imag(stemplate),2)+np.power(np.real(template)-np.real(stemplate),2))
#             dist2 = np.mean(np.abs(np.absolute(template) - np.absolute(stemplate)))
#             dist = dist2*100
#             if dist<1.25 and not np.isnan(dist):
# #                 cv2.rectangle(targetImage, ( w-swidth,h-sheight), (w,h), (255,0,0), 2)
# #                 cv2.imshow('crop', crop) 
# #                 cv2.waitKey(0) 
# #                 cv2.destroyAllWindows() 
#                 x.append(h)
#                 y.append(w)
#                 print(dist)

# for i,j in zip(x,y):
#     cv2.rectangle(targetImage, ( j-swidth,i-sheight), (j,i), (255,0,0), 2)

# cv2.imwrite("./output.jpg",targetImage)


In [ ]:
x = []
y = []


for h in range(sheight,height,2):  #skip 2
    for w in range(swidth,width,2):
        crop = targetImage[h-sheight:h,w-swidth:w]
        stemplate,contour,checker = find_dft(crop)
        size = min(50,len(stemplate))
#         stemplate_unit = np.divide(stemplate,np.mean(np.absolute(stemplate)))

        
        if len(stemplate):
            stemplate = stemplate[:size]
            miniTemplate = template[:size]
#             miniTemplate_unit = np.divide(miniTemplate,np.mean(np.absolute(miniTemplate)))
            dist2 = np.mean(np.abs(np.absolute(miniTemplate) - np.absolute(stemplate)))
            dist = dist2*100
            if dist<1.25 and not np.isnan(dist): #1.25
                x.append(h)
                y.append(w)
                crop = cv2.threshold(crop, thresh, 255, cv2.THRESH_BINARY)[1]


#                 cv2.imshow('crop', crop) 
#                 cv2.waitKey(0) 
#                 cv2.destroyAllWindows() 
#                 print(dist)


for i,j in zip(x,y):
    cv2.rectangle(targetImage, ( j-swidth,i-sheight), (j,i), (255,0,0), 2)

cv2.imwrite("./output.jpg",targetImage)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


True

In [ ]:
# crop = targetImage[224-sheight:224,394-swidth:394]


# stemplate,contour,checker = find_dft(crop)
# cv2.drawContours(crop, contours[0], -1, (0, 255, 0), 1) 


# size = min(50,len(stemplate))
# stemplate = stemplate[:size]
# miniTemplate = template[:size]
# dist2 = np.mean(np.abs(np.absolute(miniTemplate) - np.absolute(stemplate)))
# cv2.imshow('crop', crop) 
# cv2.waitKey(0) 
# cv2.destroyAllWindows() 
# print(len(stemplate))

In [ ]:
# image = cv2.imread('./sample5.png') 
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
# # gray = cv2.threshold(gray, thresh, 255, cv2.THRESH_BINARY)[1]
  
# # Find Canny edges 
# edged = cv2.Canny(gray, 80, 200) 
# cv2.imshow('Canny Edges After Contouring', edged) 
# cv2.waitKey(0) 
# cv2.destroyAllWindows() 


